In [1]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#importing package for Regular expression operations
import re 
#Package for splitting the data
from sklearn.model_selection import train_test_split 
#Package for conversion of categorical to Numerical
from sklearn.preprocessing import LabelEncoder
#Tokenization
from keras.preprocessing.text import Tokenizer 
#Add zeros or crop based on the length
from keras.preprocessing.sequence import pad_sequences 
#Sequential Neural Network
from keras.models import Sequential 
#For layers in Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D 
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


## Apply the code on spam data set available in the source code

In [3]:
#Loading the dataset
data = pd.read_csv('spam.csv', encoding='latin-1') 

In [4]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [5]:
#Keeping only the neccessary columns
data = data[['v1','v2']] 

In [6]:
#converting to lower case
data['v2'] = data['v2'].apply(lambda x: x.lower()) 
#only a-z,A-Z,0-9 would be remaining in the data, else special characters are removed
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
#Removing Retweets
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') 

In [8]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)

In [9]:
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

In [10]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

In [11]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [15]:
batch_size = 32
model = createmodel()

model.fit(X_train, Y_train, epochs =5, batch_size=batch_size, verbose = 2, validation_data=(X_test, Y_test))
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3733 samples, validate on 1839 samples
Epoch 1/5
 - 59s - loss: 0.1890 - accuracy: 0.9373 - val_loss: 0.0706 - val_accuracy: 0.9799
Epoch 2/5
 - 58s - loss: 0.0423 - accuracy: 0.9866 - val_loss: 0.0711 - val_accuracy: 0.9831
Epoch 3/5
 - 57s - loss: 0.0196 - accuracy: 0.9933 - val_loss: 0.0694 - val_accuracy: 0.9853
Epoch 4/5
 - 58s - loss: 0.0117 - accuracy: 0.9957 - val_loss: 0.0785 - val_accuracy: 0.9826
Epoch 5/5
 - 57s - loss: 0.0055 - accuracy: 0.9987 - val_loss: 0.1012 - val_accuracy: 0.9837
0.10124110566050042
0.9836868047714233
